In [ ]:
# Parameters: 
organization_id <- "440"
aggregation <- "week" # aggregation can be by day ('day'), week ('week') or month('month')
from_date <-'2017-09-01'
to_date <-'2018-12-31'

In [ ]:
library("httr")
library("jsonlite")
library("ggplot2")
library('RJSplot')

## Using API

In [ ]:
base <- "193.175.238.88:8080/api/organizations/"
call1 <- paste(base,"all/", sep="")
get_organizations_df <- as.data.frame(fromJSON(content(GET(call1), "text", encoding="UTF-8"), flatten = TRUE))
call_tweets <- paste(base,"twitter/tweets_by/organizations/", organization_id, "/?from_date=",from_date,"&to_date=",to_date,"&aggregate_by=", aggregation, sep="")
call_replies <- paste(base,"twitter/replies_to/organizations/", organization_id, "/?from_date=",from_date,"&to_date=",to_date,"&aggregate_by=", aggregation, sep="")
tweets_by = as.data.frame(fromJSON(content(GET(call_tweets), "text", encoding="UTF-8"), flatten = TRUE))
replies_to = as.data.frame(fromJSON(content(GET(call_replies), "text", encoding="UTF-8"), flatten = TRUE))
merged = merge(tweets_by, replies_to, by='labels')

In [ ]:
#plotting
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = merged, mapping = aes(as.Date(labels))) +
  stat_smooth(aes(y = values.x, color="Tweets", group=response_type.x), se = F, method = "lm", formula = y ~ poly(x, 8)) +
  stat_smooth(aes(y = values.y, color="Replies", group=response_type.y), se = F, method = "lm", formula = y ~ poly(x, 8)) +
  labs(title = "Twitter (tweets)", x = "", y = "Tweets") +
  theme(axis.text.x = element_text( size = 7, angle = 60, hjust = 0.5, vjust = 0.5),
        axis.text.y = element_text( size = 7), legend.title = element_blank())  + scale_y_continuous(labels = scales::number_format(big.mark = "")) +
scale_x_date(date_breaks = "2 weeks")